In [16]:
import pymongo
import os,csv,sys,glob
import openslide
import dsa_mongo_common_functions as dsa
import cdsa_loader_helper_functions as cdsa_helpers
import pprint
client = pymongo.MongoClient('localhost',27017)
dsa_load_errors_db = client['DSA_LoadErrors']


In [14]:
XML_LocalRoot = '/home/dagutman/devel/ADRCPathViewer_2.0/static/'

xml_files = glob.glob(XML_LocalRoot+'xmls/*/*.xml')
#print xml_files

xml_data_set = {}

for x in xml_files:
    xobj = {}
    xobj['localPath'] = x
    xobj['webPath'] = x.replace(XML_LocalRoot,'')
    slideName = os.path.basename(x).replace('.xml','')
    xobj['slideName'] = slideName
    xml_data_set[slideName] = xobj

In [17]:
bsd = { 'slideID': '222222', 'slideName': '1000676.svs'}


dsa_load_errors_db['BadSlides'].insert_one(bsd)

In [6]:
dsa_load_errors_db['BadSlides'].distinct('slideName')

[u'1000676.svs']

In [18]:
## This is specific to a given fle system and or structurme
slide_root = '/GLOBAL_SCRATCH/ADRC/'  ##Base Path for Slides
dsa_slide_db = client['ADRC_DSA_Slide_DB']  ### These need to be configured for the specific project

### To generalize this, need to describe organization, most common will be  PATIENT/STAIN_TYPE as subdirectories
subj_dir_list = [x for x in os.listdir(slide_root) if os.path.isdir(os.path.join(slide_root,x))]
print len(subj_dir_list),"Potential Patient directories were identified"

104 Potential Patient directories were identified


In [3]:
dsa_slide_db.collection_names()

[u'system.indexes', u'RawSlideData', u'DSA_Slide_Data']

In [8]:
def find_rawslide_lists( slide_root_path ):
        """project_name is passed along with the potentially more than one root image path for ndpi files"""
        slide_files = []

        slide_root_path  = slide_root_path.rstrip('/')
        print slide_root_path
        for dpath, dnames, fnames in os.walk( slide_root_path, followlinks=True):
                
                for file in fnames:
                    if '.ndpi' in file or '.svs' in file:
                                slide_files.append(dpath +'/'+file)
        print len(slide_files),"SVS or NDPI files were located"
        return slide_files

In [9]:
### So this creates a document WITHIN the current database to store raw information about the slides
## Now that I think about this, I really should not store the filename but the File Hash as I.. want to change the filenames

for sd in subj_dir_list:
    print sd,"is being processed",
    curr_svs_slide_list = find_rawslide_lists(  os.path.join(slide_root,sd)  )
    slides_processed = newly_processed = dup_slide = rescanned_slides =  0
    for sld in curr_svs_slide_list:

        slide_name = os.path.basename(sld)
        qry = dsa_slide_db['RawSlideData'].find_one( {'slide_name':slide_name})
        #print qry
        if not qry:
            fs = os.path.getsize(sld)
            #md5Checksum = dsa.md5sum(sld)
            (openslide_could_open, width, height, filesize, orig_resolution, slide_name,md5, sld_properties) = cdsa_helpers.openslide_test_file_mongo( sld, 'ndpi', client)
            if openslide_could_open:
                prep_type = 'Unknown'
                slide_metadata = { 'slide_w_path': sld, 'slide_name': slide_name, 'file_size':fs, 'width':width, 'height':height,
                                 'orig_resolution': orig_resolution, 'sld_properties': cdsa_helpers.clean_openslide_keys ( sld_properties), 'slide_md5': md5, 'prep_type': prep_type
                                 }
                dsa_slide_db['RawSlideData'].insert_one(slide_metadata)
                newly_processed +=1 
            else:
                print "UNABLE TO OPEN FILE??",sld
                ###Need to flag/load this in to an error database
        else:
            fs = os.path.getsize(sld)
            ### I had some weird fringe cases where the filesize was different.. but the filename was the same... so I needed to flag these
            ### This was the least expensive operation I had to find a "weird" file..
        slides_processed +=1        
        output = "Total Processed: %d  Newly Processed: %d Dup Slides or Already Loaded: %d  RESCANNED Slides %d" % (slides_processed, newly_processed, dup_slide, rescanned_slides )
        dsa.LinePrinter(output)


ADRC61-124 is being processed /GLOBAL_SCRATCH/ADRC/ADRC61-124
106 SVS or NDPI files were located
Total Processed: 106  Newly Processed: 0 Dup Slides or Already Loaded: 0  RESCANNED Slides 0ADRC50-10 is being processed /GLOBAL_SCRATCH/ADRC/ADRC50-10
58 SVS or NDPI files were located
Total Processed: 58  Newly Processed: 0 Dup Slides or Already Loaded: 0  RESCANNED Slides 0ADRC61-128 is being processed /GLOBAL_SCRATCH/ADRC/ADRC61-128
99 SVS or NDPI files were located
Total Processed: 99  Newly Processed: 0 Dup Slides or Already Loaded: 0  RESCANNED Slides 0ADRC50-40 is being processed /GLOBAL_SCRATCH/ADRC/ADRC50-40
59 SVS or NDPI files were located
Total Processed: 59  Newly Processed: 0 Dup Slides or Already Loaded: 0  RESCANNED Slides 0ADRC51-60 is being processed /GLOBAL_SCRATCH/ADRC/ADRC51-60
10 SVS or NDPI files were located
Total Processed: 10  Newly Processed: 0 Dup Slides or Already Loaded: 0  RESCANNED Slides 0@eaDir is being processed /GLOBAL_SCRATCH/ADRC/@eaDir
0 SVS or NDPI f

Verify failed with: (u"Can't validate JPEG for directory 0: Expected marker at 4294972165, found none",)
Verify failed with:

Openslide returned an error /GLOBAL_SCRATCH/ADRC/ADRC61-84/Tau/ADRC61-84_A15_US-tau.ndpi
Openslide returned an error /GLOBAL_SCRATCH/ADRC/ADRC61-84/Tau/ADRC61-84_A15_US-tau.ndpi
SHIT IT DIED!
UNABLE TO OPEN FILE?? /GLOBAL_SCRATCH/ADRC/ADRC61-84/Tau/ADRC61-84_A15_US-tau.ndpi
Total Processed: 104  Newly Processed: 0 Dup Slides or Already Loaded: 0  RESCANNED Slides 0ADRC62-14 is being processed /GLOBAL_SCRATCH/ADRC/ADRC62-14
1 SVS or NDPI files were located
Total Processed: 1  Newly Processed: 0 Dup Slides or Already Loaded: 0  RESCANNED Slides 0ADRC60-179 is being processed /GLOBAL_SCRATCH/ADRC/ADRC60-179
96 SVS or NDPI files were located
Total Processed: 96  Newly Processed: 0 Dup Slides or Already Loaded: 0  RESCANNED Slides 0ADRC49-04 is being processed /GLOBAL_SCRATCH/ADRC/ADRC49-04
18 SVS or NDPI files were located
Total Processed: 18  Newly Processed: 0 Dup Slides or Already Loaded: 0  RESCANNED Slides 0ADRC56-97 is being processed /GLOBAL_SCRATCH/ADRC/ADRC56-97
116 SVS or NDPI fi

 (u"Can't validate JPEG for directory 0: Expected marker at 4294972345, found none",)
Verify failed with:

Openslide returned an error /GLOBAL_SCRATCH/ADRC/ADRC51-06/BIELS/ADRC51-06_10_BIELS.ndpi
Openslide returned an error /GLOBAL_SCRATCH/ADRC/ADRC51-06/BIELS/ADRC51-06_10_BIELS.ndpi
SHIT IT DIED!
UNABLE TO OPEN FILE?? /GLOBAL_SCRATCH/ADRC/ADRC51-06/BIELS/ADRC51-06_10_BIELS.ndpi
Total Processed: 27  Newly Processed: 0 Dup Slides or Already Loaded: 0  RESCANNED Slides 0Openslide returned an error /GLOBAL_SCRATCH/ADRC/ADRC51-06/BIELS/ADRC51-06_8_BIELS.ndpi
Openslide returned an error /GLOBAL_SCRATCH/ADRC/ADRC51-06/BIELS/ADRC51-06_8_BIELS.ndpi
SHIT IT DIED!
UNABLE TO OPEN FILE?? /GLOBAL_SCRATCH/ADRC/ADRC51-06/BIELS/ADRC51-06_8_BIELS.ndpi
Total Processed: 37  Newly Processed: 0 Dup Slides or Already Loaded: 0  RESCANNED Slides 0Openslide returned an error /GLOBAL_SCRATCH/ADRC/ADRC51-06/To_Be_Sorted/ADRC51-06_8_S-M.ndpi
Openslide returned an error /GLOBAL_SCRATCH/ADRC/ADRC51-06/To_Be_Sorted/ADRC51-06_8_S-M.ndpi
SHIT IT DIED!
UNABLE TO OPEN FILE?? /GLOBAL_SCRATCH/ADRC/ADRC51-06/To_Be_Sorted/ADRC5

...

 (u"Can't validate JPEG for directory 0: Expected marker at 4294971812, found none",)
Verify failed with:

Openslide returned an error /GLOBAL_SCRATCH/ADRC/ADRC62-120/Uncategorized/ADRC62-120_A4_blank.ndpi
Openslide returned an error /GLOBAL_SCRATCH/ADRC/ADRC62-120/Uncategorized/ADRC62-120_A4_blank.ndpi
SHIT IT DIED!
UNABLE TO OPEN FILE?? /GLOBAL_SCRATCH/ADRC/ADRC62-120/Uncategorized/ADRC62-120_A4_blank.ndpi
Total Processed: 51  Newly Processed: 0 Dup Slides or Already Loaded: 0  RESCANNED Slides 0

 (u"Can't validate JPEG for directory 0: Expected marker at 4294972165, found none",)
Verify failed with:

Openslide returned an error /GLOBAL_SCRATCH/ADRC/ADRC62-120/Biels/ADRC62-120_A3_BIELS.ndpi
Openslide returned an error /GLOBAL_SCRATCH/ADRC/ADRC62-120/Biels/ADRC62-120_A3_BIELS.ndpi
SHIT IT DIED!
UNABLE TO OPEN FILE?? /GLOBAL_SCRATCH/ADRC/ADRC62-120/Biels/ADRC62-120_A3_BIELS.ndpi
Total Processed: 106  Newly Processed: 0 Dup Slides or Already Loaded: 0  RESCANNED Slides 0ADRC54-155 is being processed /GLOBAL_SCRATCH/ADRC/ADRC54-155
25 SVS or NDPI files were located
Total Processed: 25  Newly Processed: 0 Dup Slides or Already Loaded: 0  RESCANNED Slides 0Miscellaneous is being processed /GLOBAL_SCRATCH/ADRC/Miscellaneous
31 SVS or NDPI files were located
Total Processed: 31  Newly Processed: 0 Dup Slides or Already Loaded: 0  RESCANNED Slides 0ADRC60-129 is being processed /GLOBAL_SCRATCH/ADRC/ADRC60-129
112 SVS or NDPI files were located
Total Processed: 112  Newly Processed: 0 Dup Slides or Already Loaded: 0  RESCANNED Slides 0ADRC53-138 is being processed /GLOBAL_SCRATCH/ADRC/ADRC53-

 (u"Can't validate JPEG for directory 0: Expected marker at 4294972345, found none",)


In [ ]:
#dsa_slide_db['DSA_Slide_Data'].delete_many({})
#print dsa_slide_db['DSA_Slide_Data'].count()

In [ ]:
### I now want to reformat all of this data to make it more useful for DSA ...

In [24]:
## Since this is pretty easy to recreate, I'm going to empty the current database
dsa_slide_db['DSA_Slide_Data'].delete_many({}) ### DELETE THE CURRENT DSA COLLECTION SINCE I AM GOING TO UPDATE IT
all_slides = dsa_slide_db['RawSlideData'].find()


FacetSearchFields = {}
FacetSearchFields['Annotated'] = False
FacetSearchFields['Annotated'] = False


for s in all_slides:
    slide_dict = {}
    #pt_id = s['slide_w_path'].split('/')[-1]
    slideGroup = s['slide_w_path'].split('/')[-3]  ### THIS WILL ALMOST CERTAINLY CHANGE FOR EACH IMAGE SET
    pt_id = s['slide_w_path'].split('/')[-1].split('.')[0]
    
    stain_type = 'UNK'
    stain_type = s['slide_w_path'].split('/')[-2]

    slide_dict = s.copy()  ### BEWARE!!!! I WAS originally passing a reference not a copy
    slide_dict.pop('_id',None)
    
    slide_dict['pt_id'] = pt_id
    slide_dict['stain_type'] = stain_type
    ### Obfuscating the global file path so everything is relative to some base path for the archive/
    slide_dict['thumbnail_image'] = '/thumbnail/' + s['slide_w_path'].replace(slide_root,'')
    slide_dict['iip_slide_w_path'] = "/fcgi-bin/iipsrvOpenslide.fcgi?DeepZoom=" +s['slide_w_path']+'.dzi'

    slide_dict['slideLabel_image'] = '/macroImage/' + s['slide_w_path'].replace(slide_root,'')
    
    slide_dict['slide_w_path'] = '/DZIMS/' + s['slide_w_path'].replace(slide_root,'')+'.dzi'
    slide_dict['slideGroup'] = slideGroup
    slide_dict['FacetSet'] = FacetSearchFields 
    slide_dict['slideLabel_image'] = '/labelImage/' + s['slide_w_path'].replace(slide_root,'')

    slide_dict['iip_thumbnail'] =  "/fcgi-bin/iipsrvOpenslide.fcgi??FIF=" +s['slide_w_path']+'&WID=200&CVT=jpeg'
    ## See fi I can find an aperioFile to match
    
    sln = slide_dict['slide_name'].replace('.ndpi','')
    if sln in xml_data_set:

        slide_dict['HasAperioXML'] = True
        slide_dict['AperioXMLUrl'] = xml_data_set[sln]['webPath']
    else:
        slide_dict['HasAperioXML'] = False


    
    dsa_slide_db['DSA_Slide_Data'].insert_one(slide_dict)

In [23]:
dsa_slide_db['DSA_Slide_Data'].find_one({'HasAperioXML':True})

{u'AperioXMLUrl': u'xmls/ADRC60-129/ADRC60-129_1F_Abeta.xml',
 u'FacetSet': {u'Annotated': False},
 u'HasAperioXML': True,
 u'_id': ObjectId('5755e6012f9b2e52995970a5'),
 u'file_size': 1688107315,
 u'height': 81408,
 u'iip_slide_w_path': u'/fcgi-bin/iipsrvOpenslide.fcgi?DeepZoom=/GLOBAL_SCRATCH/ADRC/ADRC60-129/aBeta/ADRC60-129_1F_Abeta.ndpi.dzi',
 u'orig_resolution': 40,
 u'prep_type': u'Unknown',
 u'pt_id': u'ADRC60-129_1F_Abeta',
 u'sld_properties': {u'hamamatsu_AHEX[0]': u'74270C57AC1B78130C3FAC0B781B083FAC0D7421083FAC0F70210847A40D703F086BA83370270847A41170230043A0137027084BA01B701F0843A4116C270C47A417781B0C47A80B742B1047AC177823104FAC197833104FA8237447186FB0396CB0',
  u'hamamatsu_AHEX[0]_fluorescence': u'004F0057005B0037003F00470043003F0047004B003F004B004F004B004F00730073007F0057004B004F004F0043004F004F004B0053004700430047004B004B004F003F00470043005700470053004F00570057005F005700630073007B00830F60',
  u'hamamatsu_AHEX[0]_ploidy': u'004F0057005B0037003F00470043003F0047004B003F004B0

In [ ]:
print dsa_slide_db['DSA_Slide_Data'].distinct('slideGroup')

In [ ]:
cur= dsa_slide_db['DSA_Slide_Data'].find({'pt_id':'ADRC50-10'})
for c in cur:
    print c

In [ ]:
dsa_slide_db['DSA_Slide_Data'].find_one()